<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Bayesian_Neural_Networks_(BNNs).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the BayesianNN class
class BayesianNN(nn.Module):
    def __init__(self, input_dim, output_dim, dropout_rate=0.5):
        super(BayesianNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)  # First fully connected layer
        self.fc2 = nn.Linear(128, output_dim)  # Second fully connected layer
        self.dropout_rate = dropout_rate  # Dropout rate for Bayesian approximation

    def forward(self, x):
        x = F.relu(self.fc1(x))  # Apply ReLU activation after first layer
        x = F.dropout(x, p=self.dropout_rate, training=True)  # Apply dropout for Bayesian approximation
        x = self.fc2(x)  # Output layer
        return x

# Example usage
model = BayesianNN(input_dim=784, output_dim=10)  # Instantiate the model
input_data = torch.randn(64, 784)  # Example input (batch_size=64, input_dim=784)
output = model(input_data)

# Print the shape of the output
print(output.shape)  # Expected shape: [64, 10]